Code is taken from https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html article. Added test generator with metrics evaluation

# Loading needed libraries

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import applications
from keras import optimizers

import numpy as np
import pandas as pd

# Parameters

Dataset was splitted manually, so there are 336 train, 80 validation and 80 test images of each class 

In [30]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = './data/train'
validation_data_dir = './data/val'
test_data_dir = './data/test'

epochs = 100
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

# Data generators

In [39]:
# this is the augmentation configuration we will use for training
augmentation_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

train_generator = augmentation_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width,
                                                                 img_height),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

# this is the augmentation configuration we will use for
# testing and validation: only rescaling
rescale_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = rescale_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = rescale_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 672 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Found 160 images belonging to 2 classes.


# Training a small convnet from scratch

In [32]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [33]:
# Defining early stopping callback
earlystop = EarlyStopping(monitor='val_accuracy',
                          min_delta=0.01,
                          patience=4,
                          verbose=1,
                          restore_best_weights=True,
                          mode='max')

# Defining checkpoint callback
filepath = './baseline.hdf5'
checkpoint = ModelCheckpoint(filepath,
                             monitor='val_accuracy',
                             verbose=0,
                             save_best_only=True,
                             mode='max')

model.fit(train_generator,
          epochs=epochs,
          validation_data=validation_generator,
          callbacks=[earlystop, checkpoint])

Epoch 1/100
42/42 [==============================] - 12s 261ms/step - loss: 1.0254 - accuracy: 0.4816 - val_loss: 0.7356 - val_accuracy: 0.5000
Epoch 2/100
42/42 [==============================] - 10s 246ms/step - loss: 0.6947 - accuracy: 0.5591 - val_loss: 0.6924 - val_accuracy: 0.5250
Epoch 3/100
42/42 [==============================] - 12s 276ms/step - loss: 0.6961 - accuracy: 0.5005 - val_loss: 0.6928 - val_accuracy: 0.5813
Epoch 4/100
42/42 [==============================] - 11s 267ms/step - loss: 0.7034 - accuracy: 0.5099 - val_loss: 0.6927 - val_accuracy: 0.5250
Epoch 5/100
42/42 [==============================] - 10s 242ms/step - loss: 0.7224 - accuracy: 0.5856 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 6/100
42/42 [==============================] - 10s 244ms/step - loss: 0.6910 - accuracy: 0.5630 - val_loss: 0.7246 - val_accuracy: 0.5000
Epoch 7/100
42/42 [==============================] - 10s 240ms/step - loss: 0.6869 - accuracy: 0.5892 - val_loss: 0.6917 - val_accuracy:

In [34]:
pd.DataFrame([model.evaluate(test_generator)],
                 columns=model.metrics_names)

10/10 [==============================] - 2s 158ms/step - loss: 0.6931 - accuracy: 0.4875


,loss,accuracy
0,0.693109,0.4875


# Using the bottleneck features of a pre-trained network

In [35]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'

nb_train_samples = 336
nb_validation_samples = 80
nb_test_samples = 80

In [41]:
def save_bottlebeck_features():

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = rescale_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None, # this means our generator will only yield batches of data, no labels
        shuffle=False)# our data will be in order, so all first 1000 images will be cats, then 1000 dogs
    
    # the predict method returns the output of a model, given
    # a generator that yields batches of numpy data
    bottleneck_features_train = model.predict(
        generator, nb_train_samples // batch_size)
    np.save('bottleneck_features_train.npy', bottleneck_features_train)

    generator = rescale_datagen.flow_from_directory(validation_data_dir,
                                                    target_size=(img_width,
                                                                 img_height),
                                                    batch_size=batch_size,
                                                    class_mode=None,
                                                    shuffle=False)

    bottleneck_features_validation = model.predict(
        generator, nb_validation_samples // batch_size)
    np.save('bottleneck_features_validation.npy',
            bottleneck_features_validation)

    generator = rescale_datagen.flow_from_directory(test_data_dir,
                                                    target_size=(img_width,
                                                                 img_height),
                                                    batch_size=batch_size,
                                                    class_mode=None,
                                                    shuffle=False)

    bottleneck_features_validation = model.predict(
        generator, nb_validation_samples // batch_size)
    np.save('bottleneck_features_test.npy', bottleneck_features_validation)

In [54]:
def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')
    train_labels = np.array([0] * (nb_train_samples) + [1] *
                            (nb_train_samples))

    validation_data = np.load('bottleneck_features_validation.npy')
    validation_labels = np.array([0] * (nb_validation_samples) + [1] *
                                 (nb_validation_samples))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Defining early stopping callback
    earlystop = EarlyStopping(monitor='val_accuracy',
                              min_delta=0.01,
                              patience=8,
                              verbose=1,
                              restore_best_weights=True,
                              mode='max')

    # Defining checkpoint callback
    filepath = './bottleneck_baseline.hdf5'
    checkpoint = ModelCheckpoint(filepath,
                                 monitor='val_accuracy',
                                 verbose=0,
                                 save_best_only=True,
                                 mode='max')

    print(model.summary())

    model.fit(train_data,
              train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels),
              callbacks=[earlystop, checkpoint])
    model.save_weights(top_model_weights_path)

    test_data = np.load('bottleneck_features_test.npy')
    test_labels = np.array([0] * (nb_test_samples) + [1] *
                           (nb_test_samples))

    df = pd.DataFrame([model.evaluate(test_data, test_labels)],
                      columns=model.metrics_names)
    return df

In [55]:
save_bottlebeck_features()
metrics = train_top_model()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               2097408   
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 257       
Total params: 2,097,665
Trainable params: 2,097,665
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
42/42 [==============================] - 1s 23ms/step - loss: 1.9676 - accuracy: 0.5756 - val_loss: 0.4580 - val_accuracy: 0.7688
Epoch 2/100
42/42 [==============================] - 1s 20ms/step - loss: 0.6404 - accuracy: 0.

In [56]:
metrics

,loss,accuracy
0,0.840628,0.7875


# Fine-tuning the top layers of a a pre-trained network
To further improve our previous result, we can try to "fine-tune" the last convolutional block of the VGG16 model alongside the top-level classifier. Fine-tuning consist in starting from a trained network, then re-training it on a new dataset using very small weight updates. In our case, this can be done in 3 steps:
1. instantiate the convolutional base of VGG16 and load its weights
2. add our previously defined fully-connected model on top, and load its weights
3. freeze the layers of the VGG16 model up to the last convolutional block

In [61]:
# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'

In [73]:
# build the VGG16 network
base_model = applications.VGG16(weights='imagenet',
                                include_top=False,
                                input_shape=(150, 150, 3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

# set the first 15 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
# fine-tuning should be done with a very slow learning rate, and typically 
# with the SGD optimizer rather than an adaptative learning rate optimizer 
# such as RMSProp. This is to make sure that the magnitude of the updates 
# stays very small, so as not to wreck the previously learned features.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

Model loaded.
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 1

In [74]:
# Defining early stopping callback
earlystop = EarlyStopping(monitor='val_accuracy',
                          min_delta=0.01,
                          patience=4,
                          verbose=1,
                          restore_best_weights=True,
                          mode='max')

# Defining checkpoint callback
filepath = './vgg_ft.hdf5'
checkpoint = ModelCheckpoint(filepath,
                             monitor='val_accuracy',
                             verbose=0,
                             save_best_only=True,
                             mode='max')

# fine-tune the model
model.fit(train_generator,
          epochs=epochs,
          validation_data=validation_generator,
          callbacks=[earlystop, checkpoint])

Epoch 1/100
42/42 [==============================] - 65s 2s/step - loss: 0.4998 - accuracy: 0.7980 - val_loss: 0.5365 - val_accuracy: 0.7937
Epoch 2/100
42/42 [==============================] - 62s 1s/step - loss: 0.3973 - accuracy: 0.8299 - val_loss: 0.4092 - val_accuracy: 0.8188
Epoch 3/100
42/42 [==============================] - 62s 1s/step - loss: 0.3985 - accuracy: 0.8177 - val_loss: 0.4704 - val_accuracy: 0.7937
Epoch 4/100
42/42 [==============================] - 62s 1s/step - loss: 0.4048 - accuracy: 0.8412 - val_loss: 0.4143 - val_accuracy: 0.8188
Epoch 5/100
42/42 [==============================] - 62s 1s/step - loss: 0.3622 - accuracy: 0.8395 - val_loss: 0.3892 - val_accuracy: 0.8250
Epoch 6/100
42/42 [==============================] - 62s 1s/step - loss: 0.3520 - accuracy: 0.8423 - val_loss: 0.3942 - val_accuracy: 0.8313
Epoch 7/100
42/42 [==============================] - 62s 1s/step - loss: 0.2924 - accuracy: 0.8824 - val_loss: 0.3871 - val_accuracy: 0.8438
Epoch 8/100
4

In [75]:
pd.DataFrame([model.evaluate(test_generator)], columns=model.metrics_names)

C:\Users\Oleg\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\keras\engine\training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


,loss,accuracy
0,0.481204,0.8125


Finally, we can see that the usage of a pre-trained model is beneficial